In [1]:
import pandas as pd

df=pd.read_csv(r"C:\Users\shivamanand\Downloads\tata_aia-users (1).csv")
zbhmapping=pd.read_csv(r"C:\Users\shivamanand\Downloads\agency_new.csv")
final_map=pd.read_csv(r"C:\Users\shivamanand\Downloads\final_map.csv")

C:\Users\shivamanand\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3343: DtypeWarning: Columns (29,36,41) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [2]:
df=df.fillna(0)
df=df[['Agency' in str(x) for x in df['Department Name']]]
def split(x):
    try:
        return x.split('_')[1].split('_')[0]
    except:
        return x

df['code_generated']=df['User Group'].apply(lambda x: split(x))
df['symbol']=df['User Group'].apply(lambda x: str(x)[0])
df=df[['Employee ID','Employee Name','Supervisor ID','Designation','User Group','code_generated','symbol']]
e_users=list(df[df['symbol']=='e']['code_generated'].unique())
d={}
for i in e_users:
    d[i]=df[df['code_generated']==i]['Employee ID'].unique()
dff=pd.DataFrame.from_dict(d,orient='index')

In [9]:
reg_user_list=df['Employee ID'].unique()
reg_user_list=list(set(reg_user_list))
list_of_lists=[]
for i,user in enumerate(reg_user_list):
    try:
        list_of_lists.append([user,dff[dff.values==user].index[0]])
    except:
        list_of_lists.append([user,"empty"])
zbh_mapping1 = pd.DataFrame(list_of_lists, columns=["User","Manager"])

# zbh_mapping1.to_csv(r"Downloads\a_to_e_mapping.csv")
# a_to_e_mapping=pd.read_csv(r"C:\Users\shivamanand\Downloads\a_to_e_mapping.csv")
a_to_e_mapping=zbh_mapping1
# a_to_e_mapping.drop(a_to_e_mapping.columns[0],axis=1,inplace=True)
a_to_e_mapping.columns=['a_User','e_manager']


In [23]:
zbhmapping_eusers=zbhmapping[zbhmapping['User'].isin(e_users)]
zbhmapping_eusers.rename({'Manager':'ZBH'},axis=1,inplace=True)
zbh_mappingConcat=a_to_e_mapping.merge(zbhmapping_eusers,how='left',left_on='e_manager',right_on='User')
df_final_mapping=pd.concat([zbh_mappingConcat[['a_User','ZBH']].rename({'a_User':'User'},axis=1),zbh_mappingConcat[zbh_mappingConcat['e_manager'].isin(e_users)][['e_manager','ZBH']].rename({'e_manager':'User'},axis=1)]).drop_duplicates()
df_final_mapping

df_final_mapping=df_final_mapping.merge(df[['Employee ID','Employee Name']],how='left',left_on='ZBH',right_on='Employee ID')

In [26]:
set(df_final_mapping['ZBH'].unique())- set(df_final_mapping['Employee ID'].unique())

{'3966341', '3992108'}

In [28]:
df_final_mapping[df_final_mapping['ZBH']=='3966341']

,User,ZBH,Employee ID,Employee Name
45028,3969714,3966341,NaN,NaN


In [29]:
df_final_mapping[df_final_mapping['ZBH']=='3992108']

,User,ZBH,Employee ID,Employee Name
21690,3994803,3992108,NaN,NaN


In [30]:
df_final_mapping.drop('Employee ID',axis=1,inplace=True)

In [51]:
#match users
set(df_final_mapping['User'].unique())-set(final_map[final_map['Channel']=='Agency']['User'].unique())

{'0'}

In [55]:
#match zbh
setA=set(final_map[final_map['Channel']=='Agency'].drop_duplicates()[['User','ZH']].sort_values(by='User').apply(lambda x: (x['User'],x['ZH']) ,axis=1).unique())

In [54]:
setB=set(df_final_mapping.drop_duplicates()[['User','Employee Name']].sort_values(by='User').apply(lambda x: (x['User'],x['Employee Name']) ,axis=1).unique())

In [46]:
len(setA-setB) #not matching

59825

In [60]:
#merge check zh differences in name
df_final_mapping.merge(final_map, left_on='User', right_on='User', how='left')

,User,ZBH,Employee Name,Channel,ZH
0,0,3971085,Rajesh Devidas Warade,NaN,NaN
1,4836442,3962016,Paras Bhutani,Agency,PANKAJ GERA
2,5037409,3962940,Ananta Narayana Thakur,Agency,YOGESH ANAND
3,4892127,3961750,Prakash Bedarakota,Agency,SITAKANTA BIRESWARA MOHAPATRA
4,4974936,3973723,Anil Mehta,Agency,PANKAJ GERA
...,...,...,...,...,...
61177,4936676,3968641,MANISH TYAGI,Agency,YOGESH ANAND
61178,4672061,3979065,Manish Sheo Shanker Chaubey,Agency,Aswini Praharaj
61179,4620274,3965207,PURUSHOTHAM B,Agency,SITAKANTA BIRESWARA MOHAPATRA
61180,4935825,3963193,INDERPREET SINGH CHHATWAL,Agency,PANKAJ GERA


In [61]:
df_final_mapping['Employee Name'].unique()

array(['Rajesh Devidas Warade', 'Paras Bhutani',
       'Ananta  Narayana Thakur', 'Prakash Bedarakota', 'Anil  Mehta',
       'Vinoth AT', 'Anshuman Ganguly', 'Shyam  Narayan  Singh',
       'Yogesh Maudgil', nan, 'Subrata Shee', 'ASHOK KUMAR PRADHAN',
       'Tirtha Ranjan Paul', 'Gurprit Singh', 'Tushar Mahendrabhai Bhatt',
       'Manoj Kumar Kukreja', 'Sumit  Amit Ghosh', 'MANISH TYAGI',
       'Jasvinder Singh', 'Rahul Jankar Patil', 'PRATYUSH BOSE',
       'Manish Vinodchandra Shah', 'Pulak Sen', 'Avudaiappan  S',
       'Manik Ashok Sharma', 'Amit Sharma', 'Arindam Chakraborty',
       'Vishal Chauhan', 'Shankar Seshan', 'Anthon Suvilas Shinde',
       'RaghavendraChary MangaiahChary D', 'Saikrishna Punaati',
       'AadhiShankar U', 'Muraleekrishnan P', 'Vinodkumar Vemula',
       'Nagendra Kailash Kumar', 'Sanjeev Sri Jain', 'Niraj Kumar Singh',
       'SUSHANTA  PATRA', 'YOGESH PARIKH', 'SUBHENDU  KUMAR NAYAK',
       'Venkatesan Masilamani', 'SANDIP  KUMAR  PARIDA',
       

In [62]:
final_map['ZH'].unique()

array(['Joy Sukumar Banerjee', 'Jay  Kumar', 'Varun  Gupta',
       'Amit Nandkishore Malhotra', 'Sourabh RaviPrakash Mishra',
       'Pradeep Shailendra Tripathi', 'PIYUSH  NAHAR',
       'Achal Trilok Pathak', 'empty', 'Vineet Raj Kumar Jain',
       'Vikas Kumar Rajak', 'Purushothama Palivela Rao',
       'Pallav  Agrawal', 'Shubham PawanKumar Tongia',
       'Ramith Raghavan K', 'Madhur Kamlesh Vijaywargiya',
       'Vishwanath Shivnarayan Tata', 'Chittaranjan Chakradhar Nayak',
       'Harsh Dhananjay Jhaveri', 'Samik  Nath', 'S. Rama  Sundaram',
       'Srinivas Subramanya Rallabandi', 'SanjaySingh Sengar',
       'Brinder Singh', 'Adilhusain Siddiqui', 'Kishore V. A. S.',
       'Kailasam Vijay Chandran', 'Rakesh Malcolm Powell',
       'Mackson Joseph Mathew', 'PANKAJ GERA', 'YOGESH ANAND',
       'SITAKANTA BIRESWARA MOHAPATRA', 'Arunkumar  R  Krishnan',
       'Aswini Praharaj', 'SACHIN CHAWLA', 'Prabhat  Kumar Bhasker',
       'Praveen Dharamvir Bahl', 'Hamid  Zoha', 'Akash 

In [ ]:
#clear diff

In [69]:
# df['Supervisor ID'].apply(lambda x: str(x).split('(')[0]).unique()
# supervisorid NOT ZBH names either along with employee name.
df

,Employee ID,Employee Name,Supervisor ID,Designation,User Group,code_generated,symbol
0,4896953,A ABDULLATHIF,MANOHAR R(4036201),Senior Insurance Wealth Manager - Prime-IK,a_3961102_CH05,3961102,a
1,4982043,A ANJANA,SUJITHA R(4927026),NEW LICENSED ADVISOR - BA,a_3962657_PK02,3962657,a
2,4963665,A AROCKIA MARY,SADAIAPPAN KARTHIK(4817216),ADVISOR - BA,a_3981218_TU02,3981218,a
3,4967656,A ARUL ADAIKALARAJ,MALAYAN MUTHULAKSHMI(4931636),LIFE PLANNER,a_3962226_PY02,3962226,a
4,4736820,A BALASUNDARAM,R R RAMESHBABU(495631),LIFE PLANNER,a_3968386_MD02,3968386,a
...,...,...,...,...,...,...,...
68699,4667943,vaishali yogeshkumar vithalani,KETAN MAGANBHAI FALDU(4617848),LIFE PLANNER,a_3966512_JG03,3966512,a
68700,4582463,vikeshkumar kanubhai patel,KRUTI JIGNESH MEHTA(4178370),ADVISOR - BA,a_3977952_AH03,3977952,a
68701,4760205,vishakha balu bhosale,Agency Admin(agencyadmin),ADVISOR - BA,a_3994008_MU34,3994008,a
68702,4498750,vithoba gabaji suryawanshi,GOUD SANTOSH SHANTILAL(186953),ADVISOR - BA,a_3992817_NS02,3992817,a


In [72]:
final_map.drop_duplicates()[final_map['Channel']=='Agency']

<ipython-input-72-fb119b83f842>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  final_map.drop_duplicates()[final_map['Channel']=='Agency']


,User,Channel,ZH
7574,4836442,Agency,PANKAJ GERA
7575,5037409,Agency,YOGESH ANAND
7576,4892127,Agency,SITAKANTA BIRESWARA MOHAPATRA
7577,4974936,Agency,PANKAJ GERA
7578,4984973,Agency,Arunkumar R Krishnan
...,...,...,...
67548,4672061,Agency,Aswini Praharaj
67549,4620274,Agency,SITAKANTA BIRESWARA MOHAPATRA
67550,4935825,Agency,PANKAJ GERA
67551,4797428,Agency,Aswini Praharaj
